<a href="https://colab.research.google.com/github/alina-nnnn/Detection_person_YOLOv5/blob/main/Yolov5_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install opencv-python-headless torch torchvision
!pip install --upgrade git+https://github.com/ultralytics/yolov5.git

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [6]:
import torch
import cv2

In [7]:
def draw_detections(frame, detections, model_names, confidence_threshold=0.5):
    """
    Делает рисовку детектируемых объектов (людей) и выводит уверенность (точность) в распознании.

    Параметры:
    ----------
    frame : np.ndarray
      Кадр видео, на котором выполняется детекция
    detections : numpy.ndarray
      Массив детекций с координатами боксов, уверенностью и классом
    model_names : list
      Список имен классов модели (person - 0)
    confidence_threshold : float
      Порог уверенности для отображения детекции

    Возвращаемое значение:
    ----------------------
    frame : numpy.ndarray
      Кадр с отрисованными детекциями
    """
    for *box, conf, cls in detections:
        if int(cls) == 0 and conf >= confidence_threshold:
            x1, y1, x2, y2 = map(int, box)
            label = f'{model_names[int(cls)]} {conf:.2f}'
            text = f'{label} ({conf * 100:.1f}%)'
            color = (0, 255, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, text, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    return frame

In [8]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5m')

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 114.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 109.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 12.8s, installed 3 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-6-14 Python-3.10.12 torch-2.3.0+cu121 CPU

100%|██████████| 40.8M/40.8M [00:00<00:00, 264MB/s]

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


In [9]:
input_video = cv2.VideoCapture('crowd.mp4')

In [10]:
# Получение параметров видео
frame_width = int(input_video.get(3))
frame_height = int(input_video.get(4))
fps = int(input_video.get(cv2.CAP_PROP_FPS))

In [11]:
# Создание объекта для записи видео
out_video = cv2.VideoWriter('output_crowd.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while input_video.isOpened():
    ret, frame = input_video.read()
    if not ret:
        break

    # Преобразование изображения и выполнение детекции
    results = model(frame)

    # Получение детекций
    detections = results.xyxy[0].cpu().numpy()

    # Отрисовка объектов и точности
    frame = draw_detections(frame, detections, model.names)

    # Запись кадра в выходное видео
    out_video.write(frame)

# Освобождение ресурсов
input_video.release()
out_video.release()

In [12]:
# Скачивание файла
from google.colab import files
files.download('output_crowd.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>